# Gain Statistical Insights into Your DataTable

Woodwork provides two methods on DataTable to allow users to better understand their data: `describe` and `get_mutual_information`

In [4]:
import pandas as pd
from woodwork import DataTable

df = pd.DataFrame({
        'id': range(3),
        'first_name': ['John', 'Jane', 'James'],
        'full_name': ['Mr. John Doe', 'Doe, Mrs. Jane', 'James Brown'],
        'email': ['john.smith@example.com', None, 'team@featuretools.com'],
        'phone_number': ['5555555555', '555-555-5555', '1-(555)-555-5555'],
        'age': [33, 25, 33],
        'signup_date': [pd.to_datetime('2020-09-01')] * 3,
        'is_registered': [True, False, True],
    })

dt = DataTable(df, index='id')
dt.types

,Physical Type,Logical Type,Semantic Tag(s)
Data Column,,,
id,Int64,WholeNumber,{index}
first_name,category,Categorical,{category}
full_name,string,NaturalLanguage,{}
email,string,NaturalLanguage,{}
phone_number,string,NaturalLanguage,{}
age,Int64,WholeNumber,{numeric}
signup_date,datetime64[ns],Datetime,{}
is_registered,boolean,Boolean,{}
